# Extracting Raw Data from Unstructured Documents

In [ ]:
%pip install -r requirements.txt
# Depending upon your environment, you may need to use the above line or the one below
# !pip install -r requirements.txt

In [1]:
from openai import OpenAI
import base64
import csv
from openai_key import OPENAI_KEY
import os
import pandas as pd
import cv2
import pytesseract
from pytesseract import Output

In [ ]:
DEFAULT_PROMPT = """Please retrieve title, invoive number, issue_date, total amount and table.
    Always return your response as a valid JSON string. The format of that string should be this, 
    {
        invoice_number: RSN/18-19/4126,
        issue_date: 09/03/2019,
        total: 76900,
        table: S Description of Goods HSN/SAC GST Quantity Rate per Disc. % Amount
                Rate
                SAMSUNG S10PLUS 85171290 12 % 1 No. 68,660.71 No. 68,660.71
                512GB G975 CERAMIC BLACK
                Batch : 1 No.
    }
    If you can't find certain information like invoice_number do this invoice_number:"", similarly do it for everything, for total leave NaN if you can't find it.
    If you can't find any information from json, return empty json as {}. Do not make things up. Even if the message isn't visible, no need to write apologising messages,
    just return the JSON string.
"""

In [ ]:
client = OpenAI(api_key=OPENAI_KEY)

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
# This could also be done with os module but for the sake of maintaining order with the csv, it is done this way;
csv_file_path = 'test/gt.csv'
titles = []

with open(csv_file_path, 'r') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    for row in csv_reader:
        title = row['title']
        titles.append(title)

In [ ]:
json_strings = []

for title in titles:
    image_local = os.path.join('./test/files', title)
    image_url = f"data:image/jpeg;base64,{encode_image(image_local)}"

    response = client.chat.completions.create(
        model='gpt-4-vision-preview', 
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": DEFAULT_PROMPT},
                    {
                        "type": "image_url",
                        "image_url": {"url": image_url}
                    }
                ],
            }
        ],
        max_tokens=500,
    )

    json_string = response.choices[0].message.content
    json_string = json_string.replace("```json\n", "").replace("\n```", "").replace("```", "")
    json_strings.append(json_string)

In [ ]:
with open("predictions.json", "w") as file:
    file.write("[\n")
    file.write(",\n".join(json_strings))
    file.write("\n]\n")

In [2]:
df_1 = pd.read_json('predictions.json')

In [3]:
df_1.head()

,invoice_number,issue_date,total,table
0,KBM/15-16/S0229,31/08/2015,310693,S Description of Goods HSN/SAC GST Quantity Ra...
1,38,19/11/2020,7000,S Description of Goods HSN/SAC GST Quantity Ra...
2,S1405G202003284,07/03/2020,15576,S.No. Parts/OP code Part Labor Description HSN...
3,123456789-SA,14/09/2017,918,S Description of Goods HSN/SAC GST Quantity Ra...
4,48,18/12/2014,12100,Product Description HSN/SAC GST Quantity Rate ...


In [4]:
(df_1=='').sum()

invoice_number    5
issue_date        2
total             1
table             1
dtype: int64

In [5]:
df_2 = pd.read_csv('test/gt.csv')

In [6]:
df_2.head()

,title,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,total,total_bbox,table,table_bbox
0,fd28f5e7-fc19-4f23-8dec-9a87b40f1a15.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c4141ea0-72cc-46f8-826f-a5d80451f8e5.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,77502028-18e6-4a51-9b7d-0cc5f4a68726.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,e8d8c551-7a88-4f7d-bccf-4b205317b21e.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a7e533f9-fea3-453e-b135-6b6d9d3c63a6.jpg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_merged = df_1.combine_first(df_2)

In [8]:
df_merged.head()

,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,table,table_bbox,title,total,total_bbox
0,KBM/15-16/S0229,NaN,31/08/2015,NaN,S Description of Goods HSN/SAC GST Quantity Ra...,NaN,fd28f5e7-fc19-4f23-8dec-9a87b40f1a15.jpg,310693,NaN
1,38,NaN,19/11/2020,NaN,S Description of Goods HSN/SAC GST Quantity Ra...,NaN,c4141ea0-72cc-46f8-826f-a5d80451f8e5.jpg,7000,NaN
2,S1405G202003284,NaN,07/03/2020,NaN,S.No. Parts/OP code Part Labor Description HSN...,NaN,77502028-18e6-4a51-9b7d-0cc5f4a68726.jpg,15576,NaN
3,123456789-SA,NaN,14/09/2017,NaN,S Description of Goods HSN/SAC GST Quantity Ra...,NaN,e8d8c551-7a88-4f7d-bccf-4b205317b21e.jpg,918,NaN
4,48,NaN,18/12/2014,NaN,Product Description HSN/SAC GST Quantity Rate ...,NaN,a7e533f9-fea3-453e-b135-6b6d9d3c63a6.jpg,12100,NaN


In [9]:
def get_bounding_box(image_path, text):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    data = pytesseract.image_to_data(image, output_type=pytesseract.Output.DICT)
    for i in range(len(data['text'])):
        if data['text'][i] == text:
            x_top_left = data['left'][i]
            y_top_left = data['top'][i]
            x_bottom_right = x_top_left + data['width'][i]
            y_bottom_right = y_top_left + data['height'][i]
            return str([x_top_left, y_top_left, x_bottom_right, y_bottom_right])
    return None

In [10]:
for index, row in df_merged.iterrows():
    image_path = f"test/files/{row['title']}"
    df_merged.loc[index, 'invoice_number_bbox'] = get_bounding_box(image_path, row['invoice_number'])
    df_merged.loc[index, 'issue_date_bbox'] = get_bounding_box(image_path, row['issue_date'])
    df_merged.loc[index, 'table_bbox'] = get_bounding_box(image_path, row['table'])

/var/folders/hg/n24r9jj93w7fwkd71c03mj9h0000gn/T/ipykernel_5675/2662139823.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1130, 308, 1392, 349]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merged.loc[index, 'invoice_number_bbox'] = get_bounding_box(image_path, row['invoice_number'])
/var/folders/hg/n24r9jj93w7fwkd71c03mj9h0000gn/T/ipykernel_5675/2662139823.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[1134, 343, 1286, 384]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merged.loc[index, 'issue_date_bbox'] = get_bounding_box(image_path, row['issue_date'])
/var/folders/hg/n24r9jj93w7fwkd71c03mj9h0000gn/T/ipykernel_5675/2662139823.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise

In [13]:
df_merged.fillna('', inplace=True)

/var/folders/hg/n24r9jj93w7fwkd71c03mj9h0000gn/T/ipykernel_5675/3870340746.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_merged.fillna('', inplace=True)


In [16]:
df_merged

,invoice_number,invoice_number_bbox,issue_date,issue_date_bbox,table,table_bbox,title,total,total_bbox
0,KBM/15-16/S0229,"[1130, 308, 1392, 349]",31/08/2015,"[1134, 343, 1286, 384]",S Description of Goods HSN/SAC GST Quantity Ra...,,fd28f5e7-fc19-4f23-8dec-9a87b40f1a15.jpg,310693,
1,38,"[252, 48, 263, 56]",19/11/2020,,S Description of Goods HSN/SAC GST Quantity Ra...,,c4141ea0-72cc-46f8-826f-a5d80451f8e5.jpg,7000,
2,S1405G202003284,,07/03/2020,,S.No. Parts/OP code Part Labor Description HSN...,,77502028-18e6-4a51-9b7d-0cc5f4a68726.jpg,15576,
3,123456789-SA,"[515, 1702, 741, 1729]",14/09/2017,,S Description of Goods HSN/SAC GST Quantity Ra...,,e8d8c551-7a88-4f7d-bccf-4b205317b21e.jpg,918,
4,48,"[527, 566, 556, 586]",18/12/2014,"[529, 517, 671, 537]",Product Description HSN/SAC GST Quantity Rate ...,,a7e533f9-fea3-453e-b135-6b6d9d3c63a6.jpg,12100,
5,1,,07/09/2015,,Description Quantity Rate Options TotalIn-flig...,,4907a532-bf75-4bc4-a08c-06f9d3f00306.png,349.85,
6,2017-40,,20/07/2017,,No. Name Qty Rate Disc. Taxable Amount IGST % ...,,19d98817-caf1-4e5a-b8b2-22881ecef5d4.png,4899.36,
7,2022/000323,"[486, 171, 554, 179]",07/07/2022,,Product Description Price Quantity Tax Rate VA...,,2ec7883e-dafe-4cc3-9836-7314ace98c14.png,141.9,
8,CIV-000676 000007-1,,30/11/2019,,ITEM DESCRIPTION QUANTITY SALES PRICE DISCOUNT...,,292518c6-bfd4-4096-92a1-7df63faf0860.png,321113.52,
9,,"[0, 0, 568, 815]",09/07/2020,,,"[0, 0, 568, 815]",aacc330f-a0ad-4248-bce9-9eab83818e2b.jpg,,


In [15]:
df_merged.to_csv('predictions.csv', index=False)